# Datasets e DataLoaders no Pytorch

Demonstração do funcionamento dos Datasets e Dataloaders no Pytorch

# Datasets

No Pytorch, datasets são objetos utilizados para armazenar, indexar e e retornar elementos de um conjunto de dados.

Usualmente eles devem ter duas funções:

- `__getitem__` para retornar o elemento de um índice e 
- `__len__` para retornar o número de elementos do dataset.

## Criando um Dataset com `torch.utils.data.TensorDataset`

Se os dados e as classes forem disponibilizados em arrays do numpy ou tensores do próprio PyTorch é possível criar um dataset utilizando [torch.utils.data.TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset):

In [1]:
import numpy as np
import torch
from torch.utils.data import TensorDataset

### Dados anotados: x_data (entrada) e target (classe a que pertence)  

In [2]:
x_data = np.random.rand(20, 3)                 # 20 vetores de 3 elementos criados aleatoriamente
target = np.random.randint(0, 3, size=20)    # 20 classes 0, 1, ou 2 criadas aleatoriamente

print('dimensões de x_data:', x_data.shape)
print('dimensões de target:', target.shape)

dimensões de x_data: (20, 3)
dimensões de target: (20,)


### Passo 1: transformar em tensores torch

In [3]:
x_data = torch.FloatTensor(x_data)
target = torch.LongTensor(target)

### Passo 2: usar `TensorDataset` para criar o dataset com os tensores

In [4]:
dataset = TensorDataset(x_data, target)

Cada elemento do dataset retornará uma tupla com dois elementos:
- dado de entrada e
- a classe correspondente

In [5]:
i = 10
x, y = dataset[i]

print('dado:', x)
print('classe:', y)

dado: 
 0.9730
 0.9963
 0.1858
[torch.FloatTensor of size 3]

classe: 1


## Criando um Dataset com `torch.utils.data.Dataset`

É possível criar uma nova classe, derivada de `Dataset` para tratar casos mais complexos e especiais.

### Estrutura básica do Dataset

Vamos criar um dataset de exemplo utilizando como classe base [torch.utils.data.Dataset](http://pytorch.org/docs/master/data.html#torch.utils.data.Dataset).

Para criar uma subclasse de `Dataset` é preciso definir as funções:
- `__len__`: para retornar o tamanho do dataset, e
- `__getitem__`: para retornar um elemento de um índice dado.

A função `__len__` é chamada quando usamos o método `len(dataset)` do Python e `__getitem__` é chamada quando fazemos a indexação `dataset[i]`.

In [6]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __len__(self):
        pass
    
    def __getitem__(self):
        pass

Acima temos a estrutura básica de uma classe do Dataset, mas precisamos de dados, por isso vamos criar dados aleatórios para testar o seu funcionamento.

### Inicialização

In [7]:
class MyDataset(Dataset):
    def __init__(self):
        super(MyDataset, self).__init__()
       
        # gera 20 vetores de 3 elementos
        self.data = torch.rand(20, 3)
    
    def __len__(self):
        pass
    
    def __getitem__(self):
        pass

No código acima utilizamos o método [torch.rand](http://pytorch.org/docs/master/torch.html#torch.rand) para gerar 20 vetores de 3 elementos (ou uma matrix 20x3) no construtor da classe, que serão nossos dados. 


### Definindo `__len__` e `__getitem__`:

In [8]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
        super(MyDataset, self).__init__()
        
        # gera 20 vetores de 3 elementos
        self.data = torch.rand(20, 3)
    
    def __len__(self):
        # retorna o tamanho da primeira dimensão
        return self.data.size(0)
    
    def __getitem__(self, i):
        # retornar a i-ésima matriz 3x3 de data
        return self.data[i, :]

Nossa classe está pronta, podemos criar um objeto dela.

### Criando e testando um objeto da nossa classe

In [9]:
dataset = MyDataset()

Nosso método `__len__` deve retornar o tamanho da primeira dimensão dos dados (20), que é o tamanho de vetores de 3 elementos no dataset.

Podemos verificar se o método está correto chamando o `len()` do Python:

In [10]:
print(len(dataset))

20


Nosso método `__getitem__()` deve retornar o elemento na i-ésima posição no dataset, no nosso caso `data[i]`:

In [11]:
i = 1
print(dataset[i])
# é equivalente à:
print(dataset.data[i])


 0.0512
 0.5284
 0.0842
[torch.FloatTensor of size 3]


 0.0512
 0.5284
 0.0842
[torch.FloatTensor of size 3]



### Incluindo o rótulo para gerar o dado anotado: entrada e rótulo (classe)

Um dataset também deve contém o rótulo ou classe a qual o dado pertence, a seguir incluiremos classes ao exemplo anterior.

In [12]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
        super(MyDataset, self).__init__()
        
        # gera 20 vetores de 3 elementos
        self.data = torch.rand(20, 3)
        # gera as 20 elementos contendo classes 0, 1 ou 2
        self.classes = (torch.rand(20)*3).type(torch.LongTensor)
    
    def __len__(self):
        # retorna o tamanho da primeira dimensão
        return self.data.size(0)
    
    def __getitem__(self, i):
        # retornar a i-ésima matriz 3x3 de data
        return (self.data[i, :], self.classes[i])

Agora nosso método `__getitem__` retorna uma tupla (dado, classe).

In [13]:
dataset = MyDataset()

data, label = dataset[1]

print('dado:', data)
print('classe:', label)

dado: 
 0.3950
 0.6465
 0.2399
[torch.FloatTensor of size 3]

classe: 2


### Conclusão

O caso apresentado neste tópico é muito simples, apresentando os conceitos básicos da estrutura que o Pytorch utiliza para representar datasets. A classe torch.utils.data.Dataset é usada para casos complexos em que é preciso, por exemplo, carregar o dataset de arquivos e quando o gerenciamento dos dados não é tão trivial.

# DataLoaders

Um DataLoader ([torch.utils.data.DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader)) combina um Dataset e um Sampler (divide os dados em batches). 

O DataLoader permite que os dados possam ser processados na forma de "mini-batches". A cada nova chamada do objeto criado com o DataLoader, um novo conjunto de dados é retornado. O DataLoader é a ferramenta do PyTorch para implementar o treinamento do gradiente descendente por "mini-batches". O treinamento por mini-batches possui duas grandes vantagens:
- Implementa o gradiente descendente estocástico via mini-batch, que acelera o treinamento;
- Permite que os dados do mini-batch a serem otimizados caibam na memória (normalmente da GPU).


## Criando um DataLoader

Vamos utilizar o dataset já criado anteriormente para ser a fonte de dados do nosso DataLoader.

In [14]:
print('tamanho do dataset: ', len(dataset))
print('amostra 12:', dataset[12])

tamanho do dataset:  20
amostra 12: (
 0.4787
 0.9966
 0.1468
[torch.FloatTensor of size 3]
, 0)


Agora podemos criar o DataLoader com o dataset

In [15]:
from torch.utils.data import DataLoader

data_loader = DataLoader(dataset, 
                         batch_size=5,           # tamanho do mini-batch de dados
                         shuffle=False)          # se for True, embaralha os dados no inicio de cada iteração

## Iterando sobre o DataLoader

Podemos iterar sobre o DataLoader utilizando um `for`

In [16]:
batch_n = 0
n_samples = 0
for data in data_loader:
    print('batch ', batch_n)
    batch_n += 1
    n_samples += len(data[0])
    
print('tamanho do DataLoader', len(data_loader))
print('tamanho do dataset', n_samples)

batch  0
batch  1
batch  2
batch  3
tamanho do DataLoader 4
tamanho do dataset 20


O tamanho do DataLoader é 4, pois temos batches de tamanho 5 e 20 dados no dataloader.

Agora podemos utilizar os dados do DataLoader:

In [17]:
batch_n = 0

for data in data_loader:
    # separa a tupla em dados e classes
    data_batch, targets_batch = data
    
    print('dimensão do batch de dados {}:   {}'.format(batch_n, data_batch.size()))
    print('dimensão do batch de classes {}: {}\n'.format(batch_n, targets_batch.size()))
    batch_n += 1


dimensão do batch de dados 0:   torch.Size([5, 3])
dimensão do batch de classes 0: torch.Size([5])

dimensão do batch de dados 1:   torch.Size([5, 3])
dimensão do batch de classes 1: torch.Size([5])

dimensão do batch de dados 2:   torch.Size([5, 3])
dimensão do batch de classes 2: torch.Size([5])

dimensão do batch de dados 3:   torch.Size([5, 3])
dimensão do batch de classes 3: torch.Size([5])



É possível ver que os batches tem mesmo 5 dados (5 matrizes 3x3 e 5 classes)

## Exercício

Defina seu próprio DataLoader na segunda célula abaixo. Faça com que cada batch tenha 10 matrizes 3x3. Utilize o dataset já criado anteriormente.

Em seguida complete o loop que itera sobre o seu DataLoader para que seja possível plotar as matrizes de cada batch. Utilize a função plot_batch para plotar as matrizes de um batch inteiro.

In [18]:
# Código usado no exercício para plotar as imagens, NÃO É NECESSÁRIO ALTERAR
import matplotlib.pyplot as plt
import math 

def plot_batch(batch, batch_n):
    fig = plt.figure(figsize=(5,2))
    for j, mat in enumerate(batch):
        plt.subplot(1, batch.size(0), j+1)
        plt.imshow(mat.numpy(), cmap='gray')
        plt.axis('off')
    plt.suptitle('batch {} - {} elementos'.format(batch_n, batch.size(0)))
    plt.show()

In [19]:
# -- Defina aqui seu DataLoader com batches de 10 elementos cada ---
data_loader = None
    
# -- Defina aqui o loop sobre todo o DataLoader dando plot em cada batch com os dados
for data in []:
    # separe o dado das classes
    
    plot_batch([], batch_n)

- A ordem dos elementos se altera entre execuções da célula acima?
- Tente colocar o parametro `shuffle` do DataLoader como `True` e obeseve o resultado do exercício, rodando várias vezes.